# Using Astropy Units

In [1]:
%xmode minimal

Exception reporting mode: Minimal


In scientific computing, we often represent physical quantities as numbers.

In [4]:
#define your objects
distance_in_miles = 50 #distance_in_miles is the object, 50 is the property
time_in_hours = 2

#Math Analogy: it's like defining your variable, then assigning it a value

velocity_in_mph = distance_in_miles / time_in_hours
#velocity_in_mph is an object that depends on two other objects

print(velocity_in_mph)

25.0


[astropy.units]: https://docs.astropy.org/en/stable/units/index.html
[plasmapy.particles]: https://docs.plasmapy.org/en/stable/notebooks/getting_started/particles.html
[plasmapy.formulary]: https://docs.plasmapy.org/en/stable/formulary/index.html

Representing a physical quantity as a number has risks. We might unknowingly perform operations with different units, like `time_in_seconds + time_in_hours`. We might even accidentally perform operations with physically incompatible units, like `length + time`, without catching our mistake. We can avoid these problems by using a units package. Packages include specific functionality, packaged together in neat modules you can call with assigned nicknames.
 
This notebook introduces [astropy.units] with an emphasis on the functionality needed to work with [plasmapy.particles] and [plasmapy.formulary]. We typically import this subpackage as `u`.

In [5]:
import astropy.units as u 

## Contents

1. [Unit basics](#Unit-basics)
2. [Unit operations](#Unit-operations) 
3. [Unit conversations](#Unit-conversions)
4. [Detaching units and values](#Detaching-units-and-values)
5. [Equivalencies](#Equivalencies)
6. [Physical constants](#Physical-constants)
7. [Units in PlasmaPy](#Units-in-PlasmaPy)
8. [Optimizing unit operations](#Optimizing-unit-operations)
9. [Physical Types](#Physical-types)

## Unit basics



We can create a physical quantity by multiplying (or dividing) a number or array with a unit.

In [12]:
distance = 60 * u.km
print(distance)

60.0 km


[Quantity]: https://docs.astropy.org/en/stable/api/astropy.units.Quantity.html#astropy.units.Quantity

This operation creates a [Quantity]: a number, sequence, or array that has been assigned a physical unit. 

The type() function tells you the class an object belongs to. In this case, `distance` is a Quantity as defined by the units package.

In [6]:
type(distance)

astropy.units.quantity.Quantity

[Quantity]: https://docs.astropy.org/en/stable/api/astropy.units.Quantity.html#astropy.units.Quantity

We can create [Quantity] objects with compound units, such as miles per hour.

In [8]:
88 * u.imperial.mile / u.hour

<Quantity 88. mi / h>

[Quantity]: https://docs.astropy.org/en/stable/api/astropy.units.Quantity.html#astropy.units.Quantity

We can even create [Quantity] objects that are explicitly dimensionless.

In [9]:
3 * u.dimensionless_unscaled

<Quantity 3.>

[Quantity]: https://docs.astropy.org/en/stable/api/astropy.units.Quantity.html#astropy.units.Quantity

We can also create a [Quantity] by assigning units to a NumPy array or a list. This gives every object in the array (or list) the same unit.

In [6]:
import numpy as np

np.array([2.5, 3.2, 1.1]) * u.kg

<Quantity [2.5, 3.2, 1.1] kg>

In [9]:
my_array = np.array([2, 3, 4]) * u.m / u.s

[Quantity]: https://docs.astropy.org/en/stable/api/astropy.units.Quantity.html#astropy.units.Quantity
[numpy.ndarray]: https://numpy.org/doc/stable/reference/generated/numpy.ndarray.html

[Quantity] objects behave very similarly to NumPy arrays because [Quantity] is a subclass of [numpy.ndarray]. Objects in a numpy array can be called by their index number, with the pattern: `my_array[index]`. 

Change the cell below from Markdown to Code, and call the first object in the array `my_array`. You will see it calls both the numerical values as well as the units.

my_array[0] #call the first object in the array

## Unit operations

[Quantity]: https://docs.astropy.org/en/stable/api/astropy.units.Quantity.html#astropy.units.Quantity

Operations between [Quantity] objects handle unit conversions automatically. We can add [Quantity] objects together as long as their units have the same physical type.

In [10]:
1 * u.m + 25 * u.cm

<Quantity 1.25 m>

Try: In the line above, what is the result if you switch the units and add 1 cm and 25 m?

Units get handled automatically during operations like multiplication, division, and exponentiation.

In [14]:
time = 120 * u.min

velocity = distance / time
print(velocity)

0.5 km / min


In [15]:
area = distance**2
print(area)

3600.0 km2


Attempting an operation between physically incompatible units gives us an error, which we can use to find bugs in our code.

In [16]:
3 * u.m + 3 * u.s

UnitConversionError: Can only apply 'add' function to quantities with compatible dimensions

You can similarly perform operations between Numpy arrays.

In [17]:
#The Balmer series is a series of lines in the Hydrogen emission spectrum, in this case listed with their wavelengths. 
balmer_series = [656.279, 486.135, 434.0472, 410.1734] * u.nm
Hα = balmer_series[0] #call different values by changing the index number
print(Hα)

656.279 nm


Numpy has a lot of useful functions for dealing with data sets. For example, np.max() finds the largest number in the array.

In [18]:
np.max(balmer_series)

<Quantity 656.279 nm>

[NumPy]: https://numpy.org/
[SciPy]: https://scipy.org/

[Quantity]: https://docs.astropy.org/en/stable/api/astropy.units.Quantity.html#astropy.units.Quantity
[Quantity objects lose their units with some operations]: https://docs.astropy.org/en/stable/known_issues.html#quantities-lose-their-units-with-some-operations

Most frequently encountered [NumPy] and [SciPy] (another package with useful science applications) functions can be used with [Quantity] objects.  However, [Quantity objects lose their units with some operations].  

Computing the frequency instead of the wavelength
$$f = \frac{c}{\lambda}$$

In [19]:
from astropy.constants import c

In [20]:
#c.to("nm/s")
bs_freq=c.to("nm/s")/balmer_series
print(bs_freq)
bs_freq=bs_freq.to("Hz")
print(bs_freq)

[4.56806416e+14 6.16685608e+14 6.90690916e+14 7.30892003e+14] 1 / s
[4.56806416e+14 6.16685608e+14 6.90690916e+14 7.30892003e+14] Hz


## Unit conversions

[Quantity]: https://docs.astropy.org/en/stable/api/astropy.units.Quantity.html#astropy.units.Quantity
[to]: https://docs.astropy.org/en/stable/api/astropy.units.Quantity.html#astropy.units.Quantity.to

The [to] method allows us to convert a [Quantity] to different units of the same physical type. This method accepts strings that represent a unit (including compound units) or a unit object. Specifically, [to] is an attribute of a Quantity, which means it is called by the pattern object.attribute(). 

In [25]:
#object.to()
velocity.to("m/s")

<Quantity 8.33333333 m / s>

In [26]:
velocity.to(u.m / u.s)

<Quantity 8.33333333 m / s>

[Quantity]: https://docs.astropy.org/en/stable/api/astropy.units.Quantity.html#astropy.units.Quantity
[si]: https://docs.astropy.org/en/stable/api/astropy.units.Quantity.html#astropy.units.Quantity.si
[cgs]: https://docs.astropy.org/en/stable/api/astropy.units.Quantity.html#astropy.units.Quantity.cgs

The [si] and [cgs] attributes convert the [Quantity] to SI or CGS (centimeters-grams-seconds) units, respectively. 

In [27]:
velocity.si

<Quantity 8.33333333 m / s>

In [28]:
velocity.cgs

<Quantity 833.33333333 cm / s>

## Detaching units and values

[value]: https://docs.astropy.org/en/stable/api/astropy.units.Quantity.html#astropy.units.Quantity.value 
[Quantity]: https://docs.astropy.org/en/stable/api/astropy.units.Quantity.html#astropy.units.Quantity

The [value] attribute of a [Quantity] provides the number (as a NumPy scalar) or NumPy array without the unit.

In [ ]:
time.value

[unit]: https://docs.astropy.org/en/stable/api/astropy.units.Quantity.html#astropy.units.Quantity.unit
[Quantity]: https://docs.astropy.org/en/stable/api/astropy.units.Quantity.html#astropy.units.Quantity

The [unit] attribute of a [Quantity] provides the unit without the value.

In [ ]:
time.unit

## Equivalencies

[electron-volt]: https://en.wikipedia.org/wiki/Electronvolt
[Boltzmann constant]: https://en.wikipedia.org/wiki/Boltzmann_constant

Plasma scientists often use the [electron-volt] (eV) as a unit of temperature. This is a shortcut for describing the thermal energy per particle, or more accurately the temperature multiplied by the [Boltzmann constant], $k_B$. Because an electron-volt is a unit of energy rather than temperature, we cannot directly convert electron-volts to kelvin.

In [ ]:
u.eV.to("K")

[astropy.units]: https://docs.astropy.org/en/stable/units/index.html
[equivalencies]: https://docs.astropy.org/en/stable/units/equivalencies.html
[temperature_energy()]: https://docs.astropy.org/en/stable/units/equivalencies.html#temperature-energy-equivalency

To handle non-standard unit conversions, [astropy.units] allows the use of [equivalencies]. The conversion from eV to K can be done by using the [temperature_energy()] equivalency.

In [ ]:
(1 * u.eV).to("K", equivalencies=u.temperature_energy())

[dimensionless_angles()]: https://docs.astropy.org/en/stable/api/astropy.units.equivalencies.dimensionless_angles.html#dimensionless-angles

[frequency]: https://en.wikipedia.org/wiki/Frequency
[angular frequency]: https://en.wikipedia.org/wiki/Angular_frequency

Radians are treated dimensionlessly when the [dimensionless_angles()] equivalency is in effect. Note that this equivalency does not account for the multiplicative factor of $2π$ that is used when converting between [frequency] and [angular frequency].

In [ ]:
(3.2 * u.rad / u.s).to("1 / s", equivalencies=u.dimensionless_angles())

## Physical constants

[astropy.constants]: https://docs.astropy.org/en/stable/constants/index.html

We can use [astropy.constants] to access the most commonly needed physical constants.

In [ ]:
from astropy.constants import c, e, k_B

print(c)

[Constant]: https://docs.astropy.org/en/stable/api/astropy.constants.Constant.html#astropy.constants.Constant
[Quantity]: https://docs.astropy.org/en/stable/api/astropy.units.Quantity.html#astropy.units.Quantity
[u.temperature_energy()]: https://docs.astropy.org/en/stable/units/equivalencies.html#temperature-energy-equivalency

A [Constant] behaves very similarly to a [Quantity]. For example, we can use the Boltzmann constant to mimic the behavior of [u.temperature_energy()].

In [ ]:
thermal_energy_per_particle = 0.6 * u.keV
temperature = thermal_energy_per_particle / k_B
print(temperature.to("MK"))

Electromagnetic constants often need the unit system to be specified. Code within PlasmaPy uses SI units.

In [ ]:
2 * e

In [ ]:
2 * e.si

## Units in PlasmaPy

[astropy.units]: https://docs.astropy.org/en/stable/units/index.html
[plasmapy.particles]: ../../particles/index.rst
[Quantity]: https://docs.astropy.org/en/stable/api/astropy.units.Quantity.html#astropy.units.Quantity
[Particle]: ../../api/plasmapy.particles.particle_class.Particle.rst
[ParticleList]: ../../api/plasmapy.particles.particle_collections.ParticleList.rst

Now we can show some uses of [astropy.units] in PlasmaPy, starting with [plasmapy.particles]. Many of the attributes of [Particle] and [ParticleList] provide [Quantity] objects.

In [ ]:
from plasmapy.particles import Particle, ParticleList

alpha = Particle("He-4 2+")

In [ ]:
alpha.charge

In [ ]:
ions = ParticleList(["O 1+", "O 2+", "O 3+"])
ions.mass

[Quantity]: https://docs.astropy.org/en/stable/api/astropy.units.Quantity.html#astropy.units.Quantity
[plasmapy.formulary]: ../../formulary/index.rst

Similarly, [Quantity] objects are the expected inputs and outputs of most functions in [plasmapy.formulary]. We can use them to calculate some plasma parameters for a typical region of the solar corona.

In [ ]:
from plasmapy.formulary import Alfven_speed, Debye_length, gyrofrequency

In [ ]:
B = 0.01 * u.T
n = 1e15 * u.m**-3
proton = Particle("p+")

In [ ]:
Alfven_speed(B=B, density=n, ion=proton).to("km /s")

In [ ]:
gyrofrequency(B=B, particle="e-")

The `to_hz` keyword provides the frequency in hertz rather than radians per second, and accounts for the factor of $2π$.

In [ ]:
gyrofrequency(B=B, particle="e-", to_hz=True)

Formulary functions perform calculations based on SI units, but accept input arguments in other units. Temperature can be given in units of temperature (e.g., kelvin) or energy (e.g., electron-volts).

In [ ]:
Debye_length(T_e=1e6 * u.K, n_e=1e9 * u.m**-3)

In [ ]:
Debye_length(T_e=86.17 * u.eV, n_e=1e3 * u.cm**-3)

## Optimizing unit operations

[performance tips]: https://docs.astropy.org/en/stable/units/index.html#performance-tips
[astropy.units]: https://docs.astropy.org/en/stable/units/index.html

Astropy's documentation includes [performance tips] for using [astropy.units] in computationally intensive situations. For example, putting compound units in parentheses reduces the need to make multiple copies of the data.

In [ ]:
volume = 0.62 * (u.barn * u.Mpc)

## Physical types

[physical type]: https://docs.astropy.org/en/stable/units/physical_types.html
[physical_type]: https://docs.astropy.org/en/stable/api/astropy.units.UnitBase.html#astropy.units.UnitBase.physical_type
[get_physical_type()]: https://docs.astropy.org/en/stable/api/astropy.units.get_physical_type.html#astropy.units.get_physical_type

A [physical type] corresponds to physical quantities with dimensionally compatible units. Astropy has functionality that represents different physical types. These physical type objects can be accessed using either the [physical_type] attribute of a unit or [get_physical_type()].

In [ ]:
(u.m**2 / u.s).physical_type

In [ ]:
u.get_physical_type("number density")

These physical type objects can be used for dimensional analysis.

In [ ]:
energy_density = (u.J * u.m**-3).physical_type
velocity = u.get_physical_type("velocity")
print(energy_density * velocity)